# Creating Generalized Ufuncs with Numba

Numba offers the `guvectorize` to generate **generalized ufuncs** which work of input arrays with different dimensions.

In [1]:
import numpy as np
import numba

## Adding a constant to a vector

In [2]:
@numba.guvectorize(['(f8[:], f8[:], f8[:])'], '(m),()->(m)')
def vec_add_const(x, y, z):
    for i in range(x.shape[0]):
        z[i] = x[i] + y[0]

In [3]:
x = np.arange(10.0)
z = vec_add_const(x, 2.0)
print(x, z, sep='\n')

[0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
[ 2.  3.  4.  5.  6.  7.  8.  9. 10. 11.]


In [4]:
x = np.arange(10.0).reshape(2, 5)
z = vec_add_const(x, 2.0)
print(x, z, sep='\n')

[[0. 1. 2. 3. 4.]
 [5. 6. 7. 8. 9.]]
[[ 2.  3.  4.  5.  6.]
 [ 7.  8.  9. 10. 11.]]


In [5]:
x = np.arange(10.0).reshape(2, 5)
y = np.array([1., 2.])
z = vec_add_const(x, y)
print(x, z, sep='\n')

[[0. 1. 2. 3. 4.]
 [5. 6. 7. 8. 9.]]
[[ 1.  2.  3.  4.  5.]
 [ 7.  8.  9. 10. 11.]]


## Matrix Vector Multiplication

In [6]:
@numba.guvectorize(['(f8[:, :], f8[:], f8[:])'], '(m,n),(n)->(m)')
def mat_vec_mult(x, y, z):
    for i in range(x.shape[0]):
        d = 0.0
        for j in range(x.shape[1]):
            d += x[i, j] * y[j]
        z[i] = d

In [7]:
A = np.arange(9.0).reshape(3, 3)
x = np.array([1., 2., 3.])
z = mat_vec_mult(A, x)
print(A, z, sep='\n\n')

[[0. 1. 2.]
 [3. 4. 5.]
 [6. 7. 8.]]

[ 8. 26. 44.]


In [8]:
A = np.arange(27.0).reshape(3, 3, 3)
x = np.array([1., 2., 3])
z = mat_vec_mult(A, x)
print(A, z, sep='\n\n')

[[[ 0.  1.  2.]
  [ 3.  4.  5.]
  [ 6.  7.  8.]]

 [[ 9. 10. 11.]
  [12. 13. 14.]
  [15. 16. 17.]]

 [[18. 19. 20.]
  [21. 22. 23.]
  [24. 25. 26.]]]

[[  8.  26.  44.]
 [ 62.  80.  98.]
 [116. 134. 152.]]


## Matrix-Matrix Multiplication

In [9]:
@numba.guvectorize(['(f8[:, :], f8[:, :], f8[:, :])'], '(m,n),(n,k)->(m, k)')
def mat_mul(x, y, z):
    for i in range(x.shape[0]):
        for j in range(y.shape[1]):
            d = 0.0
            for k in range(x.shape[1]):
                d += x[i, k] * y[k, j]
            z[i, j] = d

In [11]:
A = np.arange(9.0).reshape(3, 3)
B = np.arange(9.0, 24.0).reshape(3, 5)
C = mat_mul(A, B)
C_numpy = A @ B
print(C, C_numpy, sep='\n\n')

[[ 52.  55.  58.  61.  64.]
 [178. 190. 202. 214. 226.]
 [304. 325. 346. 367. 388.]]

[[ 52.  55.  58.  61.  64.]
 [178. 190. 202. 214. 226.]
 [304. 325. 346. 367. 388.]]
